<a href="https://colab.research.google.com/github/mash1negun/ppa-for-da/blob/AddingFile/students/%D0%9C%D0%B0%D0%BC%D0%B0%D1%88%D0%B5%D0%B2_%D0%97%D0%B0%D0%B2%D1%83%D1%80/recommender_system(%D0%A0%D0%B5%D0%B2%D1%8C%D1%8E%20%D0%BD%D0%B0%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%83%20%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D1%8F%20%D0%9A%D0%BB%D0%B8%D0%BC%D0%B5%D0%BD%D0%BA%D0%BE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Библиотека Surprise

Библиотека Surprise используется для создания рекомендательных систем. Рекомендательная система (англ. recommender system) - это подкласс системы фильтрации информации, которая стремится предсказать некоторый рейтинг или уровень предпочтения, который пользователь соотнесёт с конкретным элементом (например, товаром, фильмом, книгой и т.д.).

**Коллаборативная фильтрация**, совместная фильтрация (англ. collaborative filtering) — метод, использующий известные предпочтения (оценки) группы пользователей для прогнозирования неизвестных предпочтений другого пользователя. Его основное допущение состоит в следующем: те, кто одинаково оценивал какие-либо предметы в прошлом, склонны давать похожие оценки другим предметам и в будущем. Например, с помощью коллаборативной фильтрации музыкальное приложение способно прогнозировать, какая музыка понравится пользователю, имея неполный список его предпочтений (симпатий и антипатий).

В данном туториале рассмотрим построение рекомендательной системы основанной на коллаборативной фильтрации.


In [12]:
! pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

import os
import csv

# Загрузим датасет с рейтингами фильмов
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
!ls

--2021-11-24 22:29:22--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.2’

ml-latest-small.zip 100%[===================>] 955.28K  4.57MB/s    in 0.2s    

2021-11-24 22:29:23 (4.57 MB/s) - ‘ml-latest-small.zip.2’ saved [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/links.csv  
replace ml-latest-small/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/tags.csv  
replace ml-latest-small/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/ratings.csv  
replace ml-latest-small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflati

Сформируем датасет

In [2]:
def load_dataset():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('ml-latest-small/ratings.csv', reader=reader)

    movieID_to_name = {}
    with open('ml-latest-small/movies.csv', newline='', encoding='ISO-8859-1') as csvfile:
            movie_reader = csv.reader(csvfile)
            next(movie_reader)
            for row in movie_reader:
                movieID = int(row[0])
                movie_name = row[1]
                movieID_to_name[movieID] = movie_name
    
    return (ratings_dataset, movieID_to_name)

dataset, movieID_to_name = load_dataset()

Небольшая утилита для получения названия фильма

In [3]:
def getMovieName(movieID):
  if int(movieID) in movieID_to_name:
    return movieID_to_name[int(movieID)]
  else:
      return ""

Разделим выборку на тренировочную и тестовую

In [4]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(dataset, test_size=0.25)

Импортируем и обучим модель

In [5]:
from surprise import SVD
svd = SVD()
svd.fit(trainset)

Воспользуемся тестовой выборкой и рассчитаем среднеквадратическую ошибку

In [13]:
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8787


0.8787301512114029

Порекомендуем пользователю фильм, предположив, что он оценит его больше чем на 4 балла

In [7]:
for prediction in predictions:
  if prediction.est >= 4:
    print("Можем порекомендовать пользователю {name} фильм {movie}".format(name=prediction.uid, movie=getMovieName(prediction.iid)))
    break

Можем порекомендовать пользователю 340 фильм Crimson Tide (1995)


# Резюме
В данном туториале была построена и обучена рекомендательная система основанная на коллаборативной фильтрации.

# Источники

1.  [Surprise’ documentation](https://surprise.readthedocs.io/en/stable/)
2.  [Wiki Recommender system](https://en.wikipedia.org/wiki/Recommender_system)
3.  [How you can build simple recommender systems with Surprise.](https://towardsdatascience.com/how-you-can-build-simple-recommender-systems-with-surprise-b0d32a8e4802)
4.  [Youtube - How To Use The Surprise Library For Recommendation Engines](https://www.youtube.com/watch?v=fEd1p8-3S7w)
5.  [Build a Recommendation Engine With Collaborative Filtering](https://realpython.com/build-recommendation-engine-collaborative-filtering/#the-dataset)

# Ревью (Мамашев Завур)

Туториал был пройден, но 13 блок с кодом не работал из-за отсутствия import accuracy.